# 参考サイト
[Qiita記事](https://qiita.com/takubb/items/fd972f0ac3dba909c293)

## 準備

In [1]:
# ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 13.1 MB/s 
     |████████████████████████████████| 7.6 MB 52.9 MB/s 
     |████████████████████████████████| 182 kB 72.5 MB/s 


In [3]:
!apt install aptitude swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl lib

In [4]:
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y

git is already installed at the requested version (1:2.17.1-1ubuntu0.13)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.21)
xz-utils is already installed at the requested version (5.2.2-1.3ubuntu0.1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.13)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.21)
xz-utils is already installed at the requested version (5.2.2-1.3ubuntu0.1)
The following NEW packages will be installed:
  file libmagic-mgc{a} libmagic1{a} libmecab-dev libmecab2{a} mecab mecab-ipadic{a} mecab-ipadic-utf8 mecab-jumandic{a} mecab-jumandic-utf8{a} mecab-utils{a} 
The following packages will be REMOVED:
  libnvidia-common-460{u} 
0 packages upgraded, 11 newly installed, 1 to remove and 5 not upgraded.
Need to get 29.3 MB of archives. After unpacking 282 MB will be used.
Get: 1 

In [5]:
!pip install mecab-python3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 14.6 MB/s 


## ライブラリ、データセットの読み込み

In [6]:
import pandas as pd
import torch
import numpy as np
import os
import time
import datetime
import pytz

dt = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))

# ベースとなるディレクトリ
BASE_DIR = "/content/drive/My Drive/hate-speech-detection-nishika/"
DATA_PATH = os.path.join(BASE_DIR, "data/raw")    # 対象データの保存ディレクトリ
MODEL_PATH = os.path.join(
    BASE_DIR, 
    f"data/predicted/submission_{dt.year}{str(dt.month).zfill(2)}{str(dt.day).zfill(2)}_BERT_PyTorch"
    )  # モデルを保存するディレクトリ

print(MODEL_PATH)

# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

/content/drive/My Drive/hate-speech-detection-nishika/data/predicted/submission_20221123_BERT_PyTorch
使用デバイス： cuda:0


In [7]:
dir = "/content/drive/My Drive/hate-speech-detection-nishika/data/raw/"

train_raw = pd.read_csv(f"{dir}train.csv")
test_raw = pd.read_csv(f"{dir}test.csv")

train = train_raw.copy()
test = test_raw.copy()
print("train size:", train.shape)
display(train.head())
print("test size:", test.shape)
display(test.head())

train size: (5256, 4)


,id,source,text,label
0,80074aa43,news4vip,まともに相手されてない人との関係なんて\nそんな大事にするものか？,0
1,6378fea6b,livejupiter,最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!,0
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ\n甘えるな,1
3,e76638295,livejupiter,よくよく思えば川上は配布にしたらとんでもなく有能だよな\nガチャから引いたら圧倒的歓喜レベルやで,0
4,51e4036bf,newsplus,押井は原作レイプの専門家だから\n原作マンガの真意を誤解させることに関してはプロだが\nそれ...,0


test size: (3223, 3)


,id,source,text
0,001026808,news4vip,上でも言ったけどオタクレベルの知識求めてる訳じゃない\nただ囲碁やります！って人が誰1人プロ...
1,00465ac96,livejupiter,たとえば、黒人なんかは、生物学的欠陥はないのに、文化的要因で、悪循環に陥り、実力をつけられず...
2,004674725,livejupiter,そうなんやろなあ色々と勿体ない感じしたわ\n終わり方と黒幕キャラは好きやったで\n\nちなワ...
3,00474460f,news4vip,法的というか自治体ごとにバラバラの条例で定めてるだけだからな\n普通の淫行条例だと「青少年に...
4,004a7525c,newsplus,別のジャーナリストの感想として言われてるので客観的な事実とは言えないけど、\n現地は不測の事...


In [8]:
# train.csvとtest.csvを縦結合
df_all = pd.concat([train, test], axis=0)
df_all

,id,source,text,label
0,80074aa43,news4vip,まともに相手されてない人との関係なんて\nそんな大事にするものか？,0.0
1,6378fea6b,livejupiter,最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!,0.0
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ\n甘えるな,1.0
3,e76638295,livejupiter,よくよく思えば川上は配布にしたらとんでもなく有能だよな\nガチャから引いたら圧倒的歓喜レベルやで,0.0
4,51e4036bf,newsplus,押井は原作レイプの専門家だから\n原作マンガの真意を誤解させることに関してはプロだが\nそれ...,0.0
...,...,...,...,...
3218,ffc4647ac,news4vip,１人がいいのか？\nなんで変なのと同棲したのか…\nなにがしたいんだ…,NaN
3219,ffc6554ba,newsplus,ロシアもだなあ\n元々北朝鮮はロシアの工作で作られた国だから,NaN
3220,ffd3b29c2,newsplus,クネが国境に拡声器を設置して昼も夜も北の悪口鳴らしてんだとよ\nお互い当たらないように大砲撃...,NaN
3221,ffd3c69b6,news4vip,当然って言い方が腹立つんだよなあ\r\nその時点で何か男より優位に立ちたいみたいな感じがして...,NaN


# 前処理
- 改行文字等を削除

In [9]:
def text_preprocess(df):
  new_text = []

  for line in df["text"]:
      line = line.strip("\n")
      line = line.replace("\n", "")
      line = line.rstrip("\u3000")
      line = line.replace("\u3000", "")
      new_text.append(line)

  df["text"] = new_text
  return df

In [10]:
df_all = text_preprocess(df_all)
df_all.head(3)

,id,source,text,label
0,80074aa43,news4vip,まともに相手されてない人との関係なんてそんな大事にするものか？,0.0
1,6378fea6b,livejupiter,最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!,0.0
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ甘えるな,1.0


In [11]:
# データの抽出（numpy.ndarray）
sentences = df_all.text.values
labels = df_all.label.values

## データの前処理

In [12]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 13.5 MB/s 


In [13]:
# 1. BERT Tokenizerを用いて単語分割・IDへ変換
## Tokenizerの準備
from transformers import BertJapaneseTokenizer

In [14]:
!pip install fugashi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 583 kB 15.1 MB/s 


In [15]:
!pip install ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.4 MB 13.7 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=89cf09f5435d17d1ea4652db81075bcff7d19fda65ee4511ccde188b0c435167
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [16]:
# Load pre-trained tokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

In [17]:
## テスト実行
for i in range(3):
  # 元文章
  print(' Original: ', sentences[i])
  # Tokenizer
  print('Tokenized: ', tokenizer.tokenize(sentences[i]))
  # Token-id
  print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[i])))
  print("=" * 20)

 Original:  まともに相手されてない人との関係なんてそんな大事にするものか？
Tokenized:  ['まとも', 'に', '相手', 'さ', 'れ', 'て', 'ない', '人', 'と', 'の', '関係', 'なんて', 'そんな', '大事', 'に', 'する', 'もの', 'か', '?']
Token IDs:  [23135, 7, 1879, 26, 20, 16, 80, 53, 13, 5, 633, 15060, 4799, 15872, 7, 34, 120, 29, 2935]
 Original:  最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!
Tokenized:  ['最近', 'は', 'ア', '##ヘ', '##ア', '##ヘ', 'Q', '##S', 'マン', 'や', 'ない', '?', 'イ', '##イ', '!(', '・', '[UNK]', '・', ')', '##+', '1', '-', '0', '(', '・', 'A', '・', ')', 'イ', '##ク', '##ナイ', '!']
Token IDs:  [5233, 9, 43, 28962, 28483, 28962, 2984, 28583, 1490, 49, 80, 2935, 88, 28478, 21567, 35, 1, 35, 24, 29676, 17, 61, 518, 23, 35, 192, 35, 24, 88, 28488, 4114, 679]
 Original:  日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ甘えるな
Tokenized:  ['日本人', 'として', '生まれ', 'て', 'も', '無', '##能', 'な', '低', '学歴', 'って', '分かっ', 'たら', '日本人', 'の', '権利', '剥奪', 'し', 'て', '追放', 'す', 'べき', 'やろ', '甘', '##える', 'な']
Token IDs:  [2839, 50, 1115, 16, 28, 348, 28742, 18, 837, 26486, 6172, 8056, 3318, 2

In [18]:
# 最大単語数の確認
max_len = []
# 1文づつ処理
for sent in sentences:
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))
# 最大の値を確認
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')

最大単語数:  97
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


In [19]:
train = text_preprocess(train)
print("train size:", train.shape)
train

train size: (5256, 4)


,id,source,text,label
0,80074aa43,news4vip,まともに相手されてない人との関係なんてそんな大事にするものか？,0
1,6378fea6b,livejupiter,最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!,0
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ甘えるな,1
3,e76638295,livejupiter,よくよく思えば川上は配布にしたらとんでもなく有能だよなガチャから引いたら圧倒的歓喜レベルやで,0
4,51e4036bf,newsplus,押井は原作レイプの専門家だから原作マンガの真意を誤解させることに関してはプロだがそれ以外には...,0
...,...,...,...,...
5251,42b5f86b0,news4vip,車じゃなくてもよくない？ケーブル網を張り巡らせてリフトみたいなのを付けるとか,0
5252,402ce15d9,livejupiter,左やぞ？出すなら下水流,0
5253,0739a9fcb,newsplus,日本によって、けんけんガクガクの議論を持たらされた韓国は被害者日本人がしっかり考えないと、、...,0
5254,d496c7dc8,news4vip,ゴムボート買って、沖まで漕いで行ったら？魚の血を塗っておけばサメが食べてくれるよ,0


In [20]:
# データの抽出（numpy.ndarray）
sentences = train.text.values
labels = train.label.values

In [21]:
input_ids = []
attention_masks = []

# 1文づつ処理
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = max(max_len), # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True, # Attention maskの作成
                        return_tensors = 'pt', #  Pytorch tensorsで返す
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention_maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# tenosor型に変換
labels = torch.tensor(labels)

# 確認
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])
print("Attention Masks:", attention_masks[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  まともに相手されてない人との関係なんてそんな大事にするものか？
Token IDs: tensor([    2, 23135,     7,  1879,    26,    20,    16,    80,    53,    13,
            5,   633, 15060,  4799, 15872,     7,    34,   120,    29,  2935,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])
Attention Masks: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0

In [22]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

# 90%地点のIDを取得
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# データセットを分割
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('訓練データ数：{}'.format(train_size))
print('検証データ数: {} '.format(val_size))

# データローダーの作成
batch_size = 32

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

訓練データ数：4730
検証データ数: 526 


## 学習済みデータのロード

In [23]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinayなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [24]:
# モデルをGPUへ転送
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## モデルの訓練

In [25]:
# 最適化手法の設定
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader: # train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        loss = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels).loss # 戻り値とここを修正
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval() # 訓練モードをオフ
    val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                loss = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels).loss # 戻り値とここを修正
            val_loss += loss.item()
    return val_loss

In [26]:
# 学習の実行
max_epoch = 4
train_loss_ = []
test_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)
    test_ = train(model)
    train_loss_.append(train_)
    test_loss_.append(test_)

## 検証

In [27]:
# 検証方法の確認（1バッチ分で計算ロジックに確認）

model.eval()# 訓練モードをオフ
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

In [28]:
## 予測結果の確認
print(f'出力:{preds}')

出力:SequenceClassifierOutput(loss=None, logits=tensor([[ 5.5377, -5.1293],
        [ 5.5785, -5.0364],
        [ 5.5705, -5.0802],
        [ 5.5570, -5.1099],
        [ 5.4473, -5.0326],
        [ 4.0248, -3.1174],
        [ 3.5894, -2.7273],
        [ 5.2526, -4.8571],
        [ 5.3638, -4.8968],
        [ 5.4846, -4.9954],
        [ 5.6579, -5.1467],
        [ 4.7247, -4.1570],
        [ 5.6361, -5.1286],
        [ 4.1131, -3.3958]], device='cuda:0'), hidden_states=None, attentions=None)


In [29]:
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])

accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()

,logit_0,logit_1,pred_label,true_label
0,5.537709,-5.129333,0,0
1,5.578499,-5.036439,0,0
2,5.570465,-5.080212,0,0
3,5.557041,-5.109863,0,0
4,5.447277,-5.032632,0,0


## 予測

In [30]:
test = df_all[df_all["label"].isnull()]
print("test size:", test.shape)

# データの抽出
sentences = test.text.values
labels = test.label.values

input_ids = []
attention_masks = []

# 1文づつ処理
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = max(max_len), # 97 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maskの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# tenosor型に変換
labels = torch.tensor(labels)

# 確認
print('Original:', sentences[0])
print('Token IDs:', input_ids[0])

print("input_ids:", len(input_ids))
print("attention_masks:", len(attention_masks))
print("labels", len(labels))

# データセットクラスの作成
test_dataset = TensorDataset(input_ids, attention_masks, labels)
print("test_dataset:", len(test_dataset))

test size: (3223, 4)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original: 上でも言ったけどオタクレベルの知識求めてる訳じゃないただ囲碁やります！って人が誰1人プロ棋士わかりません、布石も知りませんなら申し訳ないけどお祈りかな
Token IDs: tensor([    2,   109,   962,  3083,    10, 11218, 25678,  2900,     5,  4125,
         2023,  7134,  3218,  4847,    80,   909, 11155,  4710,  2610,   679,
         6172,    53,    14,  3654,    17,    53,   285, 10400, 14428,  6769,
         1058,     6,  3432, 28922,    28,  4534,  6769,  1058,   737,  4482,
        29522,    80, 11218,    73, 30284, 28477,    29,    18,     3,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])
input_ids: 3223
attention_masks: 3223
labels 3223
test_dataset: 3223


In [31]:
# テストデータローダー
test_dataloader = DataLoader(
            test_dataset, 
            sampler = SequentialSampler(test_dataset), # 順番にデータを取得してバッチ化
            # batch_size = batch_size # 32
        )
print("test_dataloader:", len(test_dataloader))

preds_list = []

model.eval()# 訓練モードをオフ
for batch in test_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   

        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                      token_type_ids=None, 
                      attention_mask=b_input_mask)
        preds_list.append(preds)

test_dataloader: 3223


In [32]:
tensor_list = []
label_list = []

for i in pd.DataFrame(preds_list).logits:
  # print(i.cpu().numpy()[0])
  tensor_list.append(i.cpu().numpy()[0])
  ## np.argmaxで大き方の値を取得
  label_list.append(np.argmax(i.cpu().numpy()[0]))

# logits_df = pd.DataFrame(tensor_list, columns=['logit_0', 'logit_1'])
pred_df = pd.DataFrame(label_list, columns=['label'])

test_df = pd.concat([test[["id", "text"]], pred_df], axis=1)
test_df

,id,text,label
0,001026808,上でも言ったけどオタクレベルの知識求めてる訳じゃないただ囲碁やります！って人が誰1人プロ棋士...,0
1,00465ac96,たとえば、黒人なんかは、生物学的欠陥はないのに、文化的要因で、悪循環に陥り、実力をつけられず...,1
2,004674725,そうなんやろなあ色々と勿体ない感じしたわ終わり方と黒幕キャラは好きやったでちなワイはダークナ...,0
3,00474460f,法的というか自治体ごとにバラバラの条例で定めてるだけだからな普通の淫行条例だと「青少年に淫ら...,0
4,004a7525c,別のジャーナリストの感想として言われてるので客観的な事実とは言えないけど、現地は不測の事態が...,0
...,...,...,...
3218,ffc4647ac,１人がいいのか？なんで変なのと同棲したのか…なにがしたいんだ…,0
3219,ffc6554ba,ロシアもだなあ元々北朝鮮はロシアの工作で作られた国だから,0
3220,ffd3b29c2,クネが国境に拡声器を設置して昼も夜も北の悪口鳴らしてんだとよお互い当たらないように大砲撃ち合...,0
3221,ffd3c69b6,当然って言い方が腹立つんだよなあ\rその時点で何か男より優位に立ちたいみたいな感じがしてくる...,0


In [33]:
# 定性的に確認
test_df[test_df["label"]==1]

,id,text,label
1,00465ac96,たとえば、黒人なんかは、生物学的欠陥はないのに、文化的要因で、悪循環に陥り、実力をつけられず...,1
17,01526f4a9,だから朴チョンヒは満州国軍から日本の士官学校に留学したんだよ朴チョンヒは満州国軍籍日本人がア...,1
37,02b6dbd69,日本人が怒ってるんだよ。日本人は生まれながらの卑怯者より、裏切り者を嫌う。,1
46,03105dd4f,韓国の男ってみんな引き締まってるし銃の撃ち方知っとるし徴兵制＝悪って考え方変えて欲しいわ,1
51,0356346f3,障害者に生まれたらどうか実際身体は劣等感しかないし、知的はボーダーか完全アウトかでもかなり変...,1
...,...,...,...
3126,f7315c3f7,未成年で間違って妊娠しても責任を負えない、負わない様に殺害する、そんな風潮を推し進めたガイジ...,1
3127,f7558f189,人の権利だからね知的障害者は人じゃないよｗ,1
3147,f99569612,イスラム教徒は何故テロ団体を産み出した神(笑)を信じ続けるのかそしてそれがテロ団体を助長させ...,1
3171,fb5eabfd4,このクソ食い民族めが！(*゜∀゜)ノ二足歩行するハエと変わらんのやでチョンはww,1


In [34]:
# 出力ディレクトリがなければ作成
os.makedirs(MODEL_PATH, exist_ok=True)

# 予測結果をcsvで出力
df_submit = test_df[["id", "label"]]
df_submit.to_csv(
    os.path.join(
        MODEL_PATH, 
        f'submit_{dt.year}{str(dt.month).zfill(2)}{str(dt.day).zfill(2)}{str(dt.hour).zfill(2)}{str(dt.minute).zfill(2)}_BERT_PyTorch.csv'
        ), 
    header=True, 
    index=False
    )